In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import FuncFormatter
import ast
import json
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['axes.linewidth'] = 1
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['xtick.major.width'] = 1
matplotlib.rcParams['ytick.major.width'] = 1
matplotlib.rcParams['legend.frameon'] = False


def plot_2D_geometry_origin(ax):
    df = pd.read_csv("AngleCompare_N_new3.csv")
    row = df[df['N'] == 5000].iloc[0]
    psi_R = np.radians(json.loads(row["psi_R_deg"]))
    psi_G = np.radians(json.loads(row["psi_G_deg"]))
    norm_R = np.array(json.loads(row["norm_phi_R"]))
    norm_G = np.array(json.loads(row["norm_phi_G"]))
    theta = np.radians(row["theta_deg"])
    norm_c_R = row["norm_c_R"]
    norm_c_G = row["norm_c_G"]
    color_R = "#9850A8"
    color_G = "#0077B6"

    phi_R = np.column_stack([norm_R * np.cos(psi_R),norm_R * np.sin(psi_R)])
    phi_G = np.column_stack([norm_G * np.cos(psi_G + theta),norm_G * np.sin(psi_G + theta)])
    ax.scatter(phi_R[:,0], phi_R[:,1], s=3, alpha=0.1, color=color_R)
    ax.scatter(phi_G[:,0], phi_G[:,1],s=3, alpha=0.1, color=color_G)
    c_R = np.array([norm_c_R, 0.0])
    c_G = norm_c_G * np.array([np.cos(theta), np.sin(theta)])
    ax.arrow(0, 0, c_R[0], c_R[1],width=0.0, head_width=0.04*norm_c_R,length_includes_head=True,color=color_R, linewidth=3)
    ax.arrow(0, 0, c_G[0], c_G[1],width=0.0, head_width=0.04*norm_c_G,length_includes_head=True,color=color_G, linewidth=3)
    ax.text( c_R[0]+1, c_R[1]+3,r"$\mathbf{c}_{\mathrm{R}}$",color=color_R,fontsize=14,ha="left",va="center")
    ax.text(1.2 * c_G[0],1.2 * c_G[1],r"$\mathbf{c}_{\mathrm{G}}$",color = color_G,fontsize=14,ha="left",va="center")

    r_max = max(norm_R.max(), norm_G.max())
    if norm_c_R < 0.3 * r_max:
        ax.plot([0, r_max], [0, 0], linestyle=":", color=color_R, linewidth=2)
    if norm_c_G < 0.3 * r_max:
        ax.plot([0, r_max*np.cos(theta)], [0, r_max*np.sin(theta)], linestyle=":", color=color_G, linewidth=2)

    for r, c in [(norm_R, color_R), (norm_G, color_G)]:
        circle_max = plt.Circle((0,0), r.max(), fill=False,linestyle='--', linewidth=1, color=c, alpha=0.8)
        circle_min = plt.Circle((0,0), r.min(), fill=False, linewidth=1, linestyle='--', color=c, alpha=0.8)
        ax.add_patch(circle_max)
        ax.add_patch(circle_min)
    r0_R = np.mean(norm_R)
    r0_G = np.mean(norm_G)
    psi_R_low, psi_R_high = np.percentile(psi_R, [0, 100])
    psi_G_low, psi_G_high = np.percentile(psi_G, [0, 100])
    for psi_edge in [psi_R_low, psi_R_high]:
        ax.plot([0, r0_R*np.cos(psi_edge)],[0, r0_R*np.sin(psi_edge)],linestyle='--', linewidth=1.5,color=color_R )
    for psi_edge in [psi_G_low, psi_G_high]:
        ax.plot([0, r0_G*np.cos(psi_edge+theta)],[0, r0_G*np.sin(psi_edge+theta)],linestyle='--', linewidth=1.5,color=color_G)
    phi_R = np.linspace(psi_R_low, psi_R_high, 200)
    r = 10  
    arc_x = r * np.cos(phi_R)
    arc_y = r * np.sin(phi_R)
    ax.plot(arc_x, arc_y, color=color_R, lw=1)
    psi_mid = 0.5 * (psi_R_low + psi_R_high)
    ax.text(r*np.cos(psi_mid)-3,r*np.sin(psi_R_high)+4,rf"$\Delta\psi_R= { np.degrees(psi_R_high-psi_R_low):.1f}^\circ$",color=color_R,fontsize=12)
    
    phi_G = np.linspace(psi_G_low+theta, psi_G_high+theta, 200)
    r = 10   
    arc_x_G = r * np.cos(phi_G)
    arc_y_G = r * np.sin(phi_G)
    ax.plot(arc_x_G, arc_y_G, color=color_G, lw=1)
    psi_mid_G = 0.5 * (psi_G_low + psi_G_high)+theta
    ax.text(r0_G*np.cos(psi_G_high+theta)+2,r0_G*np.sin(psi_mid_G)-4,rf"$\Delta\psi_G= { np.degrees(psi_G_high-psi_G_low):.1f}^\circ$",color=color_G,fontsize=12)
    
    thrta_GR = np.linspace(0, theta, 200)
    r=3
    x_theta = r * np.cos(thrta_GR)
    y_theta = r * np.sin(thrta_GR)
    ax.plot(x_theta, y_theta, color='red', lw=1)
    theta_mid = 0.5 * (theta)
    ax.text(r*np.cos(theta_mid)-2,r*np.sin(theta_mid)+1,rf"$\theta$",color='red',fontsize=12)
   
    lim = 1.01 * r_max
    ax.set_xlim(-lim-0.1*r_max, lim)
    ax.set_ylim(-lim, lim+0.1*r_max)
    ax.set_aspect("equal")
    ax.set_title("(a)", fontsize=16)
    ax.legend(fontsize=11, loc="lower right")
    for spine in ["top", "right", "left", "bottom"]:
        ax.spines[spine].set_visible(False)
    theta_text1 = (rf"$N=5000$ " )
    ax.text( 0.99, 0.0, theta_text1, transform=ax.transAxes,
        fontsize=12, fontfamily='Times New Roman', ha='right', va='top',
        bbox=dict(facecolor='white', alpha=0.0, edgecolor='none') )

    ax.tick_params(left=False, bottom=False,
               labelleft=False, labelbottom=False)
    #===================Insert plot======================
    # ----------------------------------------
    df = pd.read_csv("AngleCompare_diffM_new3.csv")
    M_vals = df["M"].values
    theta_vals = df["theta_deg"].values
    axins = inset_axes(
        ax, width="35%", height="28%",  bbox_to_anchor=(0.45, 0.16, 1, 1),  
        bbox_transform=ax.transAxes,  loc="lower left", borderpad=0 )

    axins.plot(M_vals, theta_vals, 'o-',color='k',alpha=0.7, lw=1, ms=2)
    axins.plot(3000, np.degrees(theta), 'ro', ms=3, zorder=5)
    axins.plot([3000, 3000], [0, np.degrees(theta)], 'r--', lw=0.8, zorder=3)
    axins.plot([0, 3000], [np.degrees(theta), np.degrees(theta)], 'r--', lw=0.8, zorder=3)
    axins.set_xlim(0, 52000)
    axins.set_ylim(50, 130)
    axins.set_xlabel(r"$M$", fontsize=9)
    axins.set_ylabel(r"$\theta$", fontsize=9)
    axins.tick_params(axis='both', labelsize=8)
    axins.spines["top"].set_visible(False)
    axins.spines["right"].set_visible(False)


def plot_angle_distribution(ax,csv_path='AngleCompare_N_new3.csv'):
    df = pd.read_csv(csv_path)
    df["psi_R_deg"] = df["psi_R_deg"].apply(lambda x:np.array(ast.literal_eval(x)))
    df["psi_G_deg"] = df["psi_G_deg"].apply(lambda x:np.array(ast.literal_eval(x)))
    available_N = df["N"].unique()
    if not (2000 in available_N and 5000 in available_N):
        raise ValueError("The CSV file must contain entries for both N=1000 and N=5000.")
    
    row_2000 = df[df['N']==2000].iloc[0]
    row_5000 = df[df['N']==5000].iloc[0]
    angle_data={
        2000:{
            'theta_deg':row_2000['theta_deg'],
            'psi_R_deg':row_2000['psi_R_deg'],
            'psi_G_deg':row_2000['psi_G_deg'],
            'T':row_2000 ['T'], },
        5000:{
            'theta_deg':row_5000['theta_deg'],
            'psi_R_deg':row_5000['psi_R_deg'],
            'psi_G_deg':row_5000['psi_G_deg'],
            'T':row_5000 ['T'],}
        }

    color_R = {2000: "#C896C8", 5000: "#9850A8"}    
    color_G = {2000: "skyblue", 5000: "#0077B6"}    
    for N in [2000,5000]:
        d = angle_data[N]
        ax.hist(d['psi_R_deg'], bins=40, density=True, alpha=0.55, facecolor=color_R[N], edgecolor='white', linewidth=0.5, label=rf"RNN $(N={N})$")
        ax.hist(d['psi_G_deg'], bins=40, density=True, alpha=0.55, facecolor=color_G[N], edgecolor='white', linewidth=0.5, label=rf"GD $(N={N})$")

    theta_text1 = (rf"$N=2000:\theta={angle_data[2000]['theta_deg']:.1f}^\circ$ " )
    ax.text(
        0.49, 0.95, theta_text1, transform=ax.transAxes,
        fontsize=12, fontfamily='Times New Roman',
        ha='right', va='top',
        bbox=dict(facecolor='white', alpha=0.0, edgecolor='none')  )
    
    theta_text2 = (rf"$N=5000:\theta={angle_data[5000]['theta_deg']:.1f}^\circ$")
    ax.text(
        0.48, 0.88, theta_text2, transform=ax.transAxes,
        fontsize=12, fontfamily='Times New Roman',
        ha='right', va='top',
        bbox=dict(facecolor='white', alpha=0.0, edgecolor='none')  )

  
    ax.set_xlim(50, 125)
    ax.set_ylim(0.0, 0.15)
    ax.set_xticks(np.linspace(50, 125, 5))
    ax.set_yticks(np.linspace(0.0, 0.15, 5))
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.0f}"))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.2f}"))
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xlabel(r"$\psi$", fontsize=12)
    ax.set_ylabel(r"$P(\psi)$", fontsize=12)
    ax.set_title("(b)", fontsize=16)
    ax.legend(fontsize=12, loc="upper right")

def plot_norm_distribution(ax,csv_path='AngleCompare_N_new3.csv'):
    df = pd.read_csv(csv_path)
    df["norm_phi_R"] = df["norm_phi_R"].apply(lambda x:np.array(ast.literal_eval(x)))
    df["norm_phi_G"] = df["norm_phi_G"].apply(lambda x:np.array(ast.literal_eval(x)))
    available_N = df["N"].unique()
    if not (2000 in available_N and 5000 in available_N):
        raise ValueError("The CSV file must contain entries for both N=1000 and N=5000.")
    
    row_2000 = df[df['N']==2000].iloc[0]
    row_5000 = df[df['N']==5000].iloc[0]
    norm_data={
        2000:{
            'norm_phi_R':row_2000['norm_phi_R'],
            'norm_phi_G':row_2000['norm_phi_G'],},
        5000:{
            'norm_phi_R':row_5000['norm_phi_R'],
            'norm_phi_G':row_5000['norm_phi_G'], }
        }

    color_R = {2000: "#C896C8", 5000: "#9850A8"}    # RNN 紫色深浅
    color_G = {2000: "skyblue", 5000: "#0077B6"}    # GD 蓝色深浅
    for N in [2000,5000]:
        d = norm_data[N]
        ax.hist(d['norm_phi_R'], bins=20, density=True, alpha=0.55, facecolor=color_R[N], edgecolor='white', linewidth=0.3, label=rf"RNN $(N={N})$")
        ax.hist(d['norm_phi_G'], bins=20, density=True, alpha=0.55, facecolor=color_G[N], edgecolor='white', linewidth=0.3, label=rf"GD $(N={N})$")
    ax.set_xlim(23, 46)
    ax.set_ylim(0.0, 1.2)
    ax.set_xticks(np.linspace(24, 47, 5))
    ax.set_yticks(np.linspace(0.0, 1.2, 5))
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.0f}"))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.2f}"))
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xlabel(r"$\|\phi\|$", fontsize=12)
    ax.set_ylabel(r"$P(\|\phi\|)$", fontsize=12)

    ax.set_title("(c)", fontsize=16)
    ax.legend(fontsize=12, loc="upper center")

fig, axes = plt.subplots(1,3, figsize=(14,4 ), dpi=300)

plot_2D_geometry_origin(axes[0])
plot_angle_distribution(axes[1])
plot_norm_distribution(axes[2])


plt.tight_layout(pad=1.0)
plt.savefig("Attarc_Dist_new3.pdf", format="pdf")
plt.show()
     

